##### Copyright 2022 The TensorFlow Authors.


In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/video/video_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/video/video_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/video/video_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/video/video_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# Классификация видео с помощью 3D-сверточной нейронной сети

В этом руководстве демонстрируется обучение трехмерной сверточной нейронной сети (CNN) для классификации видео с использованием набора данных распознавания действий [UCF101](https://www.crcv.ucf.edu/data/UCF101.php). 3D CNN использует трехмерный фильтр для выполнения сверток. Ядро может скользить в трех направлениях, тогда как в 2D CNN оно может скользить в двух измерениях. Модель основана на работе, опубликованной в книге «Более детальный взгляд на пространственно-временные свертки для распознавания действий» (https://arxiv.org/abs/1711.11248v3) Д. Трана и др. (2017). В этом уроке вы:

- Построить входной конвейер
- Постройте 3D-модель сверточной нейронной сети с остаточными связями, используя функциональный API Keras.
- Обучите модель
- Оценить и протестировать модель

Это видеоурок по классификации является второй частью серии видеоуроков по TensorFlow. Вот еще три урока:

- [Загрузка видеоданных](https://www.tensorflow.org/tutorials/load_data/video): в этом руководстве объясняется большая часть кода, используемого в этом документе.
- [MoViNet для распознавания потоковой передачи] (https://www.tensorflow.org/hub/tutorials/movinet): ознакомьтесь с моделями MoViNet, доступными в TF Hub.
- [Перенос обучения для классификации видео с помощью MoViNet] (https://www.tensorflow.org/tutorials/video/transfer_learning_with_movinet): в этом руководстве объясняется, как использовать предварительно обученную модель классификации видео, обученную на другом наборе данных с помощью UCF- 101 набор данных.


## Настройка

Начните с установки и импорта некоторых необходимых библиотек, в том числе:
[remotezip](https://github.com/gtsystem/python-remotezip), чтобы проверить содержимое ZIP-файла, [tqdm](https://github.com/tqdm/tqdm), чтобы использовать индикатор выполнения, [ OpenCV](https://opencv.org/) для обработки видеофайлов, [einops](https://github.com/arogozhnikov/einops/tree/master/docs) для выполнения более сложных тензорных операций и [`tensorflow_docs `](https://github.com/tensorflow/docs/tree/master/tools/tensorflow_docs) для встраивания данных в блокнот Jupyter.

**Примечание**. Для запуска этого руководства используйте TensorFlow 2.10. Версии выше TensorFlow 2.10 могут работать некорректно.


In [2]:
!pip install remotezip tqdm opencv-python einops
!pip install tensorflow==2.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 48.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling te

In [3]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers

## Загрузка и предварительная обработка видеоданных

Скрытая ячейка ниже определяет вспомогательные функции для загрузки фрагмента данных из набора данных UCF-101 и загрузки его в tf.data.Dataset. Подробнее о конкретных этапах предварительной обработки можно узнать в [Учебном руководстве по загрузке видеоданных](../load_data/video.ipynb), где этот код рассматривается более подробно.

Класс FrameGenerator в конце скрытого блока является здесь самой важной утилитой. Он создает итерируемый объект, который может передавать данные в конвейер данных TensorFlow. В частности, этот класс содержит генератор Python, который загружает видеокадры вместе с их закодированной меткой. Функция генератора (`__call__`) возвращает массив кадров, созданный с помощью `frames_from_video_file`, и вектор метки, закодированный в горячем виде, связанный с набором кадров.


In [4]:
# @title


def list_files_per_class(zip_url):
    """
    Перечислите файлы в каждом классе набора данных по URL-адресу zip-архива.

         Аргументы:
           zip_url: URL-адрес, с которого можно распаковать файлы.

         Возвращаться:
           files: Список файлов в каждом из классов.
    """
    files = []
    with rz.RemoteZip(URL) as zip:
        for zip_info in zip.infolist():
            files.append(zip_info.filename)
    return files


def get_class(fname):
    """
    Получить имя класса по имени файла.

         Аргументы:
           fname: Имя файла в наборе данных UCF101.

         Возвращаться:
           Класс, которому принадлежит файл.
    """
    return fname.split("_")[-3]


def get_files_per_class(files):
    """
    Получите файлы, принадлежащие каждому классу.

         Аргументы:
           files: список файлов в наборе данных.

         Возвращаться:
           Словарь имен классов (ключ) и файлов (значения).
    """
    files_for_class = collections.defaultdict(list)
    for fname in files:
        class_name = get_class(fname)
        files_for_class[class_name].append(fname)
    return files_for_class


def download_from_zip(zip_url, to_dir, file_names):
    """
    Загрузите содержимое zip-файла по URL-адресу zip.

         Аргументы:
           zip_url: URL-адрес архива, содержащий данные.
           to_dir: каталог для загрузки данных.
           file_names: Имена файлов для загрузки.
    """
    with rz.RemoteZip(zip_url) as zip:
        for fn in tqdm.tqdm(file_names):
            class_name = get_class(fn)
            zip.extract(fn, str(to_dir / class_name))
            unzipped_file = to_dir / class_name / fn

            fn = pathlib.Path(fn).parts[-1]
            output_file = to_dir / class_name / fn
            unzipped_file.rename(
                output_file,
            )


def split_class_lists(files_for_class, count):
    """
    Возвращает список файлов, принадлежащих подмножеству данных, а также остальную часть
         файлы, которые необходимо скачать.

         Аргументы:
           files_for_class: файлы, принадлежащие определенному классу данных.
           count: количество файлов для загрузки.

         Возвращаться:
           Split_files: файлы, принадлежащие подмножеству данных.
           remainder: словарь остатка файлов, которые необходимо загрузить.
    """
    split_files = []
    remainder = {}
    for cls in files_for_class:
        split_files.extend(files_for_class[cls][:count])
        remainder[cls] = files_for_class[cls][count:]
    return split_files, remainder


def download_ufc_101_subset(zip_url, num_classes, splits, download_dir):
    """
    Загрузите подмножество набора данных UFC101 и разделите его на различные части, например
       обучение, валидация и тестирование.

       Аргументы:
         zip_url: URL-адрес архива, содержащий данные.
         num_classes: количество меток.
         splits: словарь, определяющий обучение, проверку, тестирование и т. д. (ключевое) разделение данных.
                 (значение — количество файлов на разделение).
         download_dir: Каталог для загрузки данных.
    """
    files = list_files_per_class(zip_url)
    for f in files:
        tokens = f.split("/")
        if len(tokens) <= 2:
            files.remove(f)  # Remove that item from the list if it does not have a filename

    files_for_class = get_files_per_class(files)

    classes = list(files_for_class.keys())[:num_classes]

    for cls in classes:
        new_files_for_class = files_for_class[cls]
        random.shuffle(new_files_for_class)
        files_for_class[cls] = new_files_for_class

    # Only use the number of classes you want in the dictionary
    files_for_class = {x: files_for_class[x] for x in list(files_for_class)[:num_classes]}

    dirs = {}
    for split_name, split_count in splits.items():
        print(split_name, ":")
        split_dir = download_dir / split_name
        split_files, files_for_class = split_class_lists(files_for_class, split_count)
        download_from_zip(zip_url, split_dir, split_files)
        dirs[split_name] = split_dir

    return dirs


def format_frames(frame, output_size):
    """
    Заполните и измените размер изображения из видео.

       Аргументы:
         frame: изображение, размер которого необходимо изменить и дополнить.
         output_size: размер пикселя изображения выходного кадра.

       Возвращаться:
         Отформатированный кадр с заполнением указанного выходного размера.
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame


def frames_from_video_file(video_path, n_frames, output_size=(224, 224), frame_step=15):
    """
    Создает кадры из каждого видеофайла, присутствующего в каждой категории.

       Аргументы:
         video_path: путь к файлу видео.
         n_frames: количество кадров, которые будут созданы для каждого видеофайла.
         output_size: размер пикселя изображения выходного кадра.

       Возвращаться:
         Массив кадров NumPy в форме (n_frames, высота, ширина, каналы).
    """
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    result.append(format_frames(frame, output_size))

    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            frame = format_frames(frame, output_size)
            result.append(frame)
        else:
            result.append(np.zeros_like(result[0]))
    src.release()
    result = np.array(result)[..., [2, 1, 0]]

    return result


class FrameGenerator:
    def __init__(self, path, n_frames, training=False):
        """Возвращает набор кадров со связанной с ними меткой.

        Аргументы:
          path: пути к видеофайлам.
          n_frames: количество кадров.
          training: логическое значение, позволяющее определить, создается ли набор обучающих данных.
        """
        self.path = path
        self.n_frames = n_frames
        self.training = training
        self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
        self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

    def get_files_and_class_names(self):
        video_paths = list(self.path.glob("*/*.avi"))
        classes = [p.parent.name for p in video_paths]
        return video_paths, classes

    def __call__(self):
        video_paths, classes = self.get_files_and_class_names()

        pairs = list(zip(video_paths, classes))

        if self.training:
            random.shuffle(pairs)

        for path, name in pairs:
            video_frames = frames_from_video_file(path, self.n_frames)
            label = self.class_ids_for_name[name]  # Encode labels
            yield video_frames, label

In [ ]:
URL = "https://storage.googleapis.com/thumos14_files/UCF101_videos.zip"
download_dir = pathlib.Path("./UCF101_subset/")
subset_paths = download_ufc_101_subset(URL, num_classes=10, splits={"train": 30, "val": 10, "test": 10}, download_dir=download_dir)

train :


 55%|█████▍    | 164/300 [00:32<00:58,  2.33it/s]

Create the training, validation, and test sets (`train_ds`, `val_ds`, and `test_ds`).


In [ ]:
n_frames = 10
batch_size = 8

output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths["train"], n_frames, training=True), output_signature=output_signature)


# Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths["val"], n_frames), output_signature=output_signature)
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths["test"], n_frames), output_signature=output_signature)

test_ds = test_ds.batch(batch_size)

## Создаём модель

Следующая трехмерная модель сверточной нейронной сети основана на статье [Более внимательный взгляд на пространственно-временные свертки для распознавания действий](https://arxiv.org/abs/1711.11248v3) Д. Трана и др. (2017). В статье сравниваются несколько версий 3D ResNets. Вместо того, чтобы работать с одним изображением с размерами «(высота, ширина)», как стандартные сети ResNet, они работают с объемом видео «(время, высота, ширина)». Самый очевидный подход к этой проблеме — заменить каждую 2D-свертку («layers.Conv2D») на 3D-свертку («layers.Conv3D»).

В этом руководстве используется свертка (2 + 1)D с [остаточными соединениями](https://arxiv.org/abs/1512.03385). Свертка (2 + 1)D позволяет разложить пространственные и временные измерения, создавая, таким образом, два отдельных шага. Преимущество этого подхода заключается в том, что факторизация сверток по пространственным и временным измерениям экономит параметры.

Для каждого выходного местоположения трехмерная свертка объединяет все векторы из трехмерного фрагмента объема для создания одного вектора в выходном объеме.

![3D-извилины](https://www.tensorflow.org/images/tutorials/video/3DCNN.png)

Эта операция принимает входные данные «время _ высота _ ширина _ каналы» и создает выходные данные «каналы» (при условии, что количество входных и выходных каналов одинаково. Таким образом, 3D-слой свертки с размером ядра `(3 x 3 x 3 )` потребуется весовая матрица с элементами `27 _ каналов \*\* 2`. В справочном документе обнаружено, что более эффективным и действенным подходом является факторизация свертки вместо одной трехмерной свертки для обработки измерений времени и пространства. они предложили свертку «(2+1)D», которая обрабатывает измерения пространства и времени отдельно. На рисунке ниже показаны факторизованные пространственные и временные свертки (2 + 1)D свертки.

![(2+1)D свертки](https://www.tensorflow.org/images/tutorials/video/2plus1CNN.png)

Основное преимущество этого подхода в том, что он уменьшает количество параметров. В свертке (2 + 1)D пространственная свертка принимает данные формы «(1, ширина, высота)», а временная свертка принимает данные формы «(время, 1, 1)». Например, для свертки (2 + 1)D с размером ядра `(3 x 3 x 3)` потребуются весовые матрицы размера `(9 * каналов**2) + (3 * каналов**2)`, меньше вдвое меньше, чем при полной 3D-свертке. В этом руководстве реализована (2 + 1)D ResNet18, где каждая свертка в сети заменяется сверткой (2+1)D.


In [ ]:
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

In [ ]:
class Conv2Plus1D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        """
        Последовательность сверточных слоев, которые сначала применяют операцию свертки к
        пространственные измерения, а затем и временные измерения.
        """
        super().__init__()
        self.seq = keras.Sequential(
            [
                # Spatial decomposition
                layers.Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
                # Temporal decomposition
                layers.Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding),
            ]
        )

    def call(self, x):
        return self.seq(x)

Модель ResNet состоит из последовательности остаточных блоков.
Остаточный блок имеет две ветви. Основная ветвь выполняет расчет, но через нее трудно пройти градиентам.
Остаточная ветвь обходит основные вычисления и в основном просто добавляет входные данные к выходным данным основной ветви.
Градиенты легко проходят через эту ветвь.
Следовательно, будет существовать простой путь от функции потерь к любой основной ветви остаточного блока.
Это позволяет избежать проблемы исчезающего градиента.

Создайте основную ветвь остаточного блока со следующим классом. В отличие от стандартной структуры ResNet, здесь используется собственный слой Conv2Plus1D вместо Layers.Conv2D.


In [ ]:
class ResidualMain(keras.layers.Layer):
    """
    Остаточный блок модели со сверткой, нормализацией слоев и
    функция активации, ReLU.
    """

    def __init__(self, filters, kernel_size):
        super().__init__()
        self.seq = keras.Sequential(
            [
                Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding="same"),
                layers.LayerNormalization(),
                layers.ReLU(),
                Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding="same"),
                layers.LayerNormalization(),
            ]
        )

    def call(self, x):
        return self.seq(x)

Чтобы добавить остаточную ветвь к основной, она должна иметь тот же размер. Уровень «Project» ниже предназначен для случаев, когда количество каналов изменяется в ветке. В частности, добавляется последовательность плотносвязных слоев с последующей нормализацией.


In [ ]:
class Project(keras.layers.Layer):
    """
    Спроецируйте определенные размеры тензора, когда данные проходят через разные
    фильтры по размеру и пониженная дискретизация.
    """

    def __init__(self, units):
        super().__init__()
        self.seq = keras.Sequential([layers.Dense(units), layers.LayerNormalization()])

    def call(self, x):
        return self.seq(x)

Используйте `add_residual_block`, чтобы ввести пропускное соединение между слоями модели.


In [ ]:
def add_residual_block(input, filters, kernel_size):
    """
    Добавьте в модель остаточные блоки. Если последние измерения входных данных
    и размер фильтра не совпадает, спроецируйте его так, чтобы последний размер совпадал.
    """
    out = ResidualMain(filters, kernel_size)(input)

    res = input
    # Using the Keras functional APIs, project the last dimension of the tensor to
    # match the new filter size
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)

    return layers.add([res, out])

Изменение размера видео необходимо для выполнения субдискретизации данных. В частности, понижение разрешения видеокадров позволяет модели проверять определенные части кадров, чтобы обнаружить закономерности, которые могут быть характерны для определенного действия. Посредством понижения дискретизации несущественная информация может быть отброшена. Более того, изменение размера видео позволит уменьшить размерность и, следовательно, ускорить обработку модели.


In [ ]:
class ResizeVideo(keras.layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        """
        Аргументы:
                 video: Тензорное представление видео в виде набора кадров.

               Возвращаться:
                 Уменьшенный размер видео в соответствии с новой высотой и шириной, до которых оно должно быть изменено.
        """
        # b stands for batch size, t stands for time, h stands for height,
        # w stands for width, and c stands for the number of channels.
        old_shape = einops.parse_shape(video, "b t h w c")
        images = einops.rearrange(video, "b t h w c -> (b t) h w c")
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, "(b t) h w c -> b t h w c", t=old_shape["t"])
        return videos

Используйте [функциональный API Keras](https://www.tensorflow.org/guide/keras/functional) для создания остаточной сети.


In [ ]:
input_shape = (None, 10, HEIGHT, WIDTH, 3)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(10)(x)

model = keras.Model(input, x)

In [ ]:
frames, label = next(iter(train_ds))
model.build(frames)

In [ ]:
# Visualize the model
keras.utils.plot_model(model, expand_nested=True, dpi=60, show_shapes=True)

## Обучите модель

Для этого урока выберите оптимизатор tf.keras.optimizers.Adam и функцию потерь tf.keras.losses.SparseCategoricalCrossentropy. Используйте аргумент «метрики», чтобы просмотреть точность производительности модели на каждом этапе.


In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])

Обучите модель в течение 50 эпох с помощью метода Keras Model.fit.

Примечание. Этот пример модели обучен на меньшем количестве точек данных (300 обучающих и 100 проверочных примеров), чтобы обеспечить разумное время обучения для этого руководства. Более того, обучение этой примерной модели может занять более часа.


In [ ]:
history = model.fit(x=train_ds, epochs=7, validation_data=val_ds)

### Визуализируйте результаты

Создайте графики потерь и точности на наборах обучения и проверки:


In [ ]:
def plot_history(history):
    """
    Построение кривых обучения и проверки.

    Аргументы:
       история: история модели со всеми метрическими показателями
    """
    fig, (ax1, ax2) = plt.subplots(2)

    fig.set_size_inches(18.5, 10.5)

    # Plot loss
    ax1.set_title("Loss")
    ax1.plot(history.history["loss"], label="train")
    ax1.plot(history.history["val_loss"], label="test")
    ax1.set_ylabel("Loss")

    # Determine upper bound of y-axis
    max_loss = max(history.history["loss"] + history.history["val_loss"])

    ax1.set_ylim([0, np.ceil(max_loss)])
    ax1.set_xlabel("Epoch")
    ax1.legend(["Train", "Validation"])

    # Plot accuracy
    ax2.set_title("Accuracy")
    ax2.plot(history.history["accuracy"], label="train")
    ax2.plot(history.history["val_accuracy"], label="test")
    ax2.set_ylabel("Accuracy")
    ax2.set_ylim([0, 1])
    ax2.set_xlabel("Epoch")
    ax2.legend(["Train", "Validation"])

    plt.show()


plot_history(history)

## Оцените модель

Используйте Keras Model.evaluate, чтобы получить потери и точность в тестовом наборе данных.

Примечание. В примере модели в этом руководстве используется подмножество набора данных UCF101, чтобы обеспечить разумное время обучения. Точность и потери можно улучшить за счет дальнейшей настройки гиперпараметров или дополнительных обучающих данных.


In [ ]:
model.evaluate(test_ds, return_dict=True)

Чтобы дополнительно визуализировать производительность модели, используйте [матрицу путаницы](https://www.tensorflow.org/api_docs/python/tf/math/confusion_matrix). Матрица путаницы позволяет оценить эффективность модели классификации за пределами точности. Чтобы построить матрицу путаницы для этой задачи классификации нескольких классов, получите фактические значения в тестовом наборе и прогнозируемые значения.


In [ ]:
def get_actual_predicted_labels(dataset):
    """
    Создайте список фактических значений истинности и прогнозов модели.

    Аргументы:
       набор данных: повторяемая структура данных, такая как набор данных TensorFlow, с функциями и метками.

    Возвращаться:
       Основная истина и прогнозируемые значения для конкретного набора данных.
    """
    actual = [labels for _, labels in dataset.unbatch()]
    predicted = model.predict(dataset)

    actual = tf.stack(actual, axis=0)
    predicted = tf.concat(predicted, axis=0)
    predicted = tf.argmax(predicted, axis=1)

    return actual, predicted

In [ ]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
    cm = tf.math.confusion_matrix(actual, predicted)
    ax = sns.heatmap(cm, annot=True, fmt="g")
    sns.set(rc={"figure.figsize": (12, 12)})
    sns.set(font_scale=1.4)
    ax.set_title("Confusion matrix of action recognition for " + ds_type)
    ax.set_xlabel("Predicted Action")
    ax.set_ylabel("Actual Action")
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    ax.xaxis.set_ticklabels(labels)
    ax.yaxis.set_ticklabels(labels)

In [ ]:
fg = FrameGenerator(subset_paths["train"], n_frames, training=True)
labels = list(fg.class_ids_for_name.keys())

In [ ]:
actual, predicted = get_actual_predicted_labels(train_ds)
plot_confusion_matrix(actual, predicted, labels, "training")

In [ ]:
actual, predicted = get_actual_predicted_labels(test_ds)
plot_confusion_matrix(actual, predicted, labels, "test")

Значения точности и полноты для каждого класса также можно рассчитать с использованием матрицы путаницы.


In [ ]:
def calculate_classification_metrics(y_actual, y_pred, labels):
    """
    Рассчитайте точность и полноту модели классификации, используя основные истинные данные и
         прогнозируемые значения.

         Аргументы:
           y_actual: Ярлыки основной истины.
           y_pred: предсказанные метки.
           labels: список классификационных меток.

         Возвращаться:
           Меры точности и отзыва.
    """
    cm = tf.math.confusion_matrix(y_actual, y_pred)
    tp = np.diag(cm)  # Diagonal represents true positives
    precision = dict()
    recall = dict()
    for i in range(len(labels)):
        col = cm[:, i]
        fp = np.sum(col) - tp[i]  # Sum of column minus true positive is false negative

        row = cm[i, :]
        fn = np.sum(row) - tp[i]  # Sum of row minus true positive, is false negative

        precision[labels[i]] = tp[i] / (tp[i] + fp)  # Precision

        recall[labels[i]] = tp[i] / (tp[i] + fn)  # Recall

    return precision, recall

In [ ]:
precision, recall = calculate_classification_metrics(actual, predicted, labels)  # Test dataset

In [ ]:
precision

In [ ]:
recall